In [ ]:
import ipywidgets as widgets
from IPython.display import display
import json
from datetime import datetime
from functools import partial
import os

# Helper function for color interpolation from white to black based on progress
def get_progress_color(progress):
    shade = int(255 * (1 - progress / 100))
    return f'rgb({shade}, {shade}, {shade})'

# Layouts
layout_medium = widgets.Layout(width='200px', height='32px')
layout_button = widgets.Layout(width='200px', height='32px')
layout_label = widgets.Layout(width='100px')

# Data store
todo_items = []

# UI elements
title_input = widgets.Text(placeholder='할 일을 입력하세요', layout=layout_medium)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_medium)
due_input = widgets.DatePicker(layout=layout_medium)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_medium)

add_button = widgets.Button(description='추가', button_style='success', layout=widgets.Layout(width='410px', height='32px'))
save_btn = widgets.Button(description='저장', button_style='info', layout=layout_button)
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=layout_button)

overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, layout=widgets.Layout(width='400px', height='16px'))
overall_progress_label = widgets.Label(value='(0%)', layout=layout_label)

todo_list_box = widgets.VBox(layout=widgets.Layout(border='1px solid black', padding='10px'))

# Update functions
def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_bar.style = {'bar_color': '#ffffff'}
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    overall_progress_bar.value = avg
    color = get_progress_color(avg)
    overall_progress_bar.style = {'bar_color': color}
    overall_progress_label.value = f"({int(avg)}%)"

def refresh_list():
    todo_list_box.children = []
    for item in todo_items:
        title = widgets.Text(value=item['title'], layout=layout_medium)
        bar_color = get_progress_color(item['progress'])
        progress_bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=layout_medium, style={'bar_color': bar_color})
        progress_slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_medium)
        due_label = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else '-', layout=layout_medium)
        priority_label = widgets.Label(value=item['priority'], layout=layout_medium)

        complete_btn = widgets.Button(description='완료✅', layout=layout_button)
        delete_btn = widgets.Button(description='삭제❌', layout=layout_button)

        def on_progress_change(change, i=item, bar=progress_bar):
            i['progress'] = change['new']
            bar.value = change['new']
            bar.style = {'bar_color': get_progress_color(change['new'])}
            update_overall_progress()

        def on_complete_clicked(b, i=item):
            i['progress'] = 100
            refresh_list()
            update_overall_progress()

        def on_delete_clicked(b, i=item):
            todo_items.remove(i)
            refresh_list()
            update_overall_progress()

        progress_slider.observe(on_progress_change, names='value')
        complete_btn.on_click(partial(on_complete_clicked, i=item))
        delete_btn.on_click(partial(on_delete_clicked, i=item))

        todo_list_box.children += [widgets.GridBox([
            widgets.Label('할 일', layout=layout_label), title,
            widgets.Label('진행률', layout=layout_label), progress_bar, progress_slider,
            widgets.Label('마감일', layout=layout_label), due_label,
            widgets.Label('중요도', layout=layout_label), priority_label,
            widgets.Label('작업', layout=layout_label), complete_btn, delete_btn
        ], layout=widgets.Layout(grid_template_columns="100px 200px", grid_gap="4px"))]

# Button logic
def add_task(b):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': progress_input.value,
            'due': due_input.value,
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        refresh_list()
        update_overall_progress()

def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([
            {
                **i,
                'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None
            } for i in todo_items
        ], f)

def load_data(b):
    try:
        if os.path.exists('todo_data.json'):
            with open('todo_data.json', 'r') as f:
                raw = json.load(f)
                todo_items.clear()
                for i in raw:
                    todo_items.append({
                        'title': i['title'],
                        'progress': i['progress'],
                        'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                        'priority': i.get('priority', '보통')
                    })
                refresh_list()
                update_overall_progress()
    except:
        print("불러오기 실패")

add_button.on_click(add_task)
save_btn.on_click(save_data)
load_btn.on_click(load_data)

# Display all
display(widgets.VBox([
    widgets.Label(value='DAVID mk23 - 색상 진행률 및 폭 통일'),
    widgets.HBox([widgets.Label("할 일", layout=layout_label), title_input]),
    widgets.HBox([widgets.Label("진행률", layout=layout_label), progress_input]),
    widgets.HBox([widgets.Label("마감일", layout=layout_label), due_input]),
    widgets.HBox([widgets.Label("중요도", layout=layout_label), priority_input]),
    add_button,
    widgets.HBox([save_btn, load_btn]),
    widgets.HBox([overall_progress_bar, overall_progress_label]),
    todo_list_box
]))

refresh_list()
update_overall_progress()
